<a href="https://colab.research.google.com/github/sumanapalle065/LLM-Assign3/blob/main/LLM_Assign3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First and foremost we start by installing 'transformers' for this project.

In [17]:
!pip install transformers datasets

In [19]:
import warnings #imported the warning module to ignore the unnecessary warnings from the code.
warnings.filterwarnings('ignore')

The following are the necessary imports that are essential to develop the LLM Model. This particular dataset focuses on identifying offensive words, hate speech and derogatory language for text classification. Given the amount of data being generated on the social media everyday it is important to train models to filter offensive content to avoid harmful content. This dataset contains the labels '1' and '0' for offensive and non-offensive words respectively.

In [20]:
import pandas as llm_pds

llm_offendata = llm_pds.read_parquet("hf://datasets/valurank/offensive-multi/data/train-00000-of-00001.parquet");

In [21]:
llm_offendata.info() #getting the information of the dataset.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36593 entries, 0 to 36592
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  36593 non-null  object
 1   text    36593 non-null  object
 2   label   36593 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 857.8+ KB


In [22]:
llm_offendata.head()

,source,text,label
0,hate_speech_offensive,!!! : as a woman you shouldnt complain about ...,0
1,hate_speech_offensive,!!!!! : boy dats cold...tyga dwn bad for cuff...,1
2,hate_speech_offensive,!!!!!!! dawg!!!! : you ever fuck a bitch an...,1
3,hate_speech_offensive,!!!!!!!!! : she look like a tranny,1
4,hate_speech_offensive,!!!!!!!!!!!!! : the shit you hear about me mi...,1


In [23]:
llm_offendata.text.to_list()[0:20]

['!!!  : as a woman you shouldnt complain about cleaning up your house. ; as a man you should always take the trash out...',
 '!!!!!  : boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!',
 '!!!!!!!   dawg!!!!  : you ever fuck a bitch and she start to cry? you be confused as shit',
 '!!!!!!!!!  :  she look like a tranny',
 '!!!!!!!!!!!!!  : the shit you hear about me might be true or it might be faker than the bitch who told it to ya &;',
 '!!!!!!!!!!!!!!!!!!": the shit just blows me..claim you so faithful and down for somebody but still fucking with hoes! &;&;&;"',
 '!!!!!!": i can not just sit up and hate on another bitch .. i got too much shit going on!"',
 '!!!!&;: cause im tired of you big bitches coming for us skinny girls!!&;',
 '" ; you might not get ya bitch back ; thats that "',
 '"  :hobbies include: fighting mariam"\n\nbitch',
 '" keeks is a bitch she curves everyone " lol i walked into a conversation like this. smh',
 '" murda gang bitch its gang land "',
 

# Data Cleaning

In [24]:
import re #Import the regular expressions module to match and manipulate text patterns.
import nltk #Import the Natural Language Toolkit for various NLP tasks.
from nltk.corpus import stopwords #Import NLTK's stopwords module to filter out frequent stop words in text.
from nltk.stem import WordNetLemmatizer #Use the WordNetLemmatizer from NLTK to reduce words to their base or root form.


#Downloading the necessary NLTK data files
nltk.download('stopwords')
nltk.download('wordnet')

#Initializing the lemmatizer and the stop words.
lemm = WordNetLemmatizer()
StopWords = set(stopwords.words('english'))

def CleanText(textdata):
    #Lowercase the text tto maintain uniformity
    textdata = textdata.lower()

    #Removing the punctuation and  the special characters
    textdata = re.sub(r'[^\w\s]', '', textdata)

    #Tokenizing the text into words
    wrdsdt = textdata.split()

    #Removing stop words and lemmatized
    wrdsdt = [lemm.lemmatize(wrd) for wrd in wrdsdt if wrd not in StopWords]

    #Joining the words back into the single string
    CleanedText = ' '.join(wrdsdt)

    return CleanedText

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


The text is cleaned and processed. Regular expressions are used to search for text patterns and some of the common words which do not provide any significant meaning ('it', 'of', 'in', etc)are eliminated.

In [25]:
# Apply the clean_text method to the 'text' column
llm_offendata['text'] = llm_offendata['text'].apply(CleanText)

In [26]:
llm_offendata.text.to_list()[0:20]

['woman shouldnt complain cleaning house man always take trash',
 'boy dat coldtyga dwn bad cuffin dat hoe 1st place',
 'dawg ever fuck bitch start cry confused shit',
 'look like tranny',
 'shit hear might true might faker bitch told ya',
 'shit blow meclaim faithful somebody still fucking hoe',
 'sit hate another bitch got much shit going',
 'cause im tired big bitch coming u skinny girl',
 'might get ya bitch back thats',
 'hobby include fighting mariam bitch',
 'keeks bitch curve everyone lol walked conversation like smh',
 'murda gang bitch gang land',
 'hoe smoke loser yea go ig',
 'bad bitch thing like',
 'bitch get',
 'bitch nigga miss',
 'bitch plz whatever',
 'bitch love',
 'bitch get cut everyday b',
 'black bottle bad bitch']